# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [ ]:
from collections import defaultdict
from itertools import product

import numpy as np
from numpy.random import choice

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [6]:

def dna_to_rna(s):
    transcription_rules = {
        'A': 'A',
        'C': 'C',
        'G': 'G',
        'T': 'U'
    }
    return "".join(transcription_rules[nucleotide] for nucleotide in s)
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

AACGUGAUUUC


### Idea of solution


The solution involves defining a dictionary that maps each DNA nucleotide (A, C, G, T) to its corresponding RNA nucleotide (A, C, G, U), then using a list comprehension within the dna_to_rna function to iterate over each nucleotide in the input DNA sequence, replacing it according to the dictionary, and finally joining the resulting list of RNA nucleotides into a single string to form the RNA sequence.

### Discussion

Expected output: AACGUGAUUUC

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [24]:
from bs4 import BeautifulSoup
from pathlib import Path

FILE_PATH = Path("src") / "codon_table.html"

def get_dict():
    with open("/Users/vynguyen/Library/Application Support/tmc/vscode/mooc-data-analysis-with-python-2023-2024/part07-e01_sequence_analysis/src/codon_table.html", "r") as file:
        content = file.read()

    soup = BeautifulSoup(content, 'html.parser')
    codon_to_aa = {}
    
    pre_tag = soup.find('pre')

    lines = pre_tag.text.strip().split('\n')
    for line in lines:
        items = line.split()
        for i in range(0, len(items), 5):
            if i + 1 < len(items):
                codon = items[i]
                amino_acid = items[i + 1]
                codon_to_aa[codon] = amino_acid

    return codon_to_aa

if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)


{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', '40285)': 'UGA', '(': '63237)', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', '32109)': 'UGG', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


### Idea of solution

- File Reading: The script reads the HTML content from the specified file path.
- HTML Parsing: It uses BeautifulSoup to parse the HTML content and locate the `<pre>` tag.
- Text Processing: The text within the `<pre>` tag is stripped of leading/trailing whitespace and split into lines.
- Data Extraction: Each line is split into items. Codon and amino acid pairs are identified and added to a dictionary.
- **Result Output:** The resulting dictionary mapping codons to amino acids is printed.


### Discussion

Make sure to replace "/path/to/your/codon_table.html" with the actual path to your codon_table.html file.</br>
The result is:</br>
{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}




3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [23]:
from bs4 import BeautifulSoup

def get_dict_list():
    with open("/Users/vynguyen/Library/Application Support/tmc/vscode/mooc-data-analysis-with-python-2023-2024/part07-e01_sequence_analysis/src/codon_table.html", "r") as file:
        content = file.read()

    soup = BeautifulSoup(content, 'html.parser')
    codon_to_aa = {}
    
    pre_tag = soup.find('pre')

    lines = pre_tag.text.strip().split('\n')
    for line in lines:
        items = line.split()
        for i in range(0, len(items), 5):
            if i + 1 < len(items):
                codon = items[i]
                amino_acid = items[i + 1]
                if amino_acid not in codon_to_aa:
                    codon_to_aa[amino_acid] = []
                codon_to_aa[amino_acid].append(codon)

    return codon_to_aa

if __name__ == '__main__':
    aa_to_codon_list = get_dict_list()
    print(aa_to_codon_list)


{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UAG'], 'UGA': ['40285)'], '63237)': ['('], 'UGG': ['32109)'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


### Idea of solution

Everything is pretty much similar to the problem 2 except this part </br>
**Data Extraction**: Each line is split into items. Codon and amino acid pairs are identified. For each amino acid, the corresponding codons are added to a list in a dictionary.


### Discussion
Expected result:

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UAG'], 'UGA': ['40285)'], '63237)': ['('], 'UGG': ['32109)'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [28]:
from bs4 import BeautifulSoup
import re

CODON_TO_AA_DICT = {'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', '40285)': 'UGA', '(': '63237)', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', '32109)': 'UGG', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}

def dna_to_rna(dna_sequence):
    return dna_sequence.replace('T', 'U')

def rna_to_prot(rna_sequence):
    codon_to_aa = CODON_TO_AA_DICT
    protein_sequence = []
    
    for i in range(0, len(rna_sequence) - 2, 3):
        codon = rna_sequence[i:i+3]
        protein_sequence.append(codon_to_aa.get(codon, '?'))
    
    return ''.join(protein_sequence)

def dna_to_prot(dna_sequence):
    rna_sequence = dna_to_rna(dna_sequence)
    return rna_to_prot(rna_sequence)

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))


MISSTM*


### Idea of solution

rna_to_prot Function: Converts an RNA sequence into a protein sequence. It processes the RNA sequence in triplets (codons) and uses the codon-to-amino-acid dictionary to translate each codon to an amino acid. If a codon is not found in the dictionary, it appends a '?' to denote an unknown amino acid.</br>

dna_to_prot Function: Converts a DNA sequence to RNA and then translates the RNA to a protein sequence using the rna_to_prot function.</br>

### Discussion

Expected result:<br/>
MISSTM*


You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [39]:
from bs4 import BeautifulSoup
import re

# Regular expressions for parsing the HTML file
CODON_REGEX = r'([A-Z]{3})\s+'
AA_REGEX = r'\s+([A-Z*]{1})\s+'
FREQUENCY_REGEX = r'(\d+)\s+'

def get_probabability_dict():
    with open("/Users/vynguyen/Library/Application Support/tmc/vscode/mooc-data-analysis-with-python-2023-2024/part07-e01_sequence_analysis/src/codon_table.html", "r") as file:
        content = file.read()
    
    soup = BeautifulSoup(content, 'html.parser')

    pre_tag = soup.find('pre') 
    parsed = pre_tag.text
    
    # Extract codons, amino acids, and frequencies using regular expressions
    codons = re.findall(CODON_REGEX, parsed)
    aminoacids = re.findall(AA_REGEX, parsed)
    frequencies = [int(freq) for freq in re.findall(FREQUENCY_REGEX, parsed)]

    # Combine codons and frequencies for each amino acid
    codon_freq_dict = {}
    for i in range(0, len(codons), 4):
        codon = codons[i]
        amino_acid = aminoacids[i]
        freq = frequencies[i // 4]
        if amino_acid not in codon_freq_dict:
            codon_freq_dict[amino_acid] = {}
        codon_freq_dict[amino_acid][codon] = freq

    # Calculate probabilities
    prob_dict = {}
    for amino_acid, codon_freqs in codon_freq_dict.items():
        total_freq = sum(codon_freqs.values())
        for codon, freq in codon_freqs.items():
            prob_dict[codon] = freq / total_freq

    return prob_dict

if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    print(codon_to_prob)
    for i in range(1 + len(items) // 6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i * 6:6 + i * 6]
        ))


{'UUU': 0.8846153846153846, 'UUC': 0.11538461538461539, 'UUA': 0.15966386554621848, 'UUG': 0.01680672268907563, 'CUU': 0.3697478991596639, 'CUC': 0.01680672268907563, 'CUA': 0.3865546218487395, 'CUG': 0.05042016806722689, 'AUU': 0.6835443037974683, 'AUC': 0.0379746835443038, 'AUA': 0.27848101265822783, 'AUG': 1.0, 'GUU': 0.47058823529411764, 'GUC': 0.025210084033613446, 'GUA': 0.453781512605042, 'GUG': 0.05042016806722689}
AUA: 0.278481	AUC: 0.037975	AUG: 1.000000	AUU: 0.683544	CUA: 0.386555	CUC: 0.016807
CUG: 0.050420	CUU: 0.369748	GUA: 0.453782	GUC: 0.025210	GUG: 0.050420	GUU: 0.470588
UUA: 0.159664	UUC: 0.115385	UUG: 0.016807	UUU: 0.884615


### Idea of solution

In reverse translation, I need to account for the fact that multiple codons can encode the same amino acid, making it difficult to determine the exact codon from the amino acid alone. I start by reading and parsing the HTML file that contains codon frequencies using BeautifulSoup. I then extract codons, amino acids, and their frequencies using regex. I compile this information into a dictionary, where each amino acid maps to a list of codons and their frequencies. To calculate the probability of each codon, I sum the frequencies of all codons for each amino acid and then divide the frequency of each codon by this sum to obtain the probability.

### Discussion

The reverse dict: <br/>
{'UUU': 0.8846153846153846, 'UUC': 0.11538461538461539, 'UUA': 0.15966386554621848, 'UUG': 0.01680672268907563, 'CUU': 0.3697478991596639, 'CUC': 0.01680672268907563, 'CUA': 0.3865546218487395, 'CUG': 0.05042016806722689, 'AUU': 0.6835443037974683, 'AUC': 0.0379746835443038, 'AUA': 0.27848101265822783, 'AUG': 1.0, 'GUU': 0.47058823529411764, 'GUC': 0.025210084033613446, 'GUA': 0.453781512605042, 'GUG': 0.05042016806722689}


Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [50]:
# Define the codon probabilities and amino acid to codon mapping
CODON_PROB = {'UUU': 0.8846153846153846, 'UUC': 0.11538461538461539, 'UUA': 0.15966386554621848, 'UUG': 0.01680672268907563, 'CUU': 0.3697478991596639, 'CUC': 0.01680672268907563, 'CUA': 0.3865546218487395, 'CUG': 0.05042016806722689, 'AUU': 0.6835443037974683, 'AUC': 0.0379746835443038, 'AUA': 0.27848101265822783, 'AUG': 1.0, 'GUU': 0.47058823529411764, 'GUC': 0.025210084033613446, 'GUA': 0.453781512605042, 'GUG': 0.05042016806722689}

AA_TO_RNA = {'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UAG'], 'UGA': ['40285)'], '63237)': ['('], 'UGG': ['32109)'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


class ProteinToMaxRNA:
    
    def __init__(self):
        self.codon_to_prob = CODON_PROB
        self.amino_acid_to_max_codon = self.get_max_codon_dict()
    
    def get_max_codon_dict(self):
        # Map each amino acid to the most probable codon
        aa_to_max_codon = {}
        for aa, codons in AA_TO_RNA.items():
            max_prob = 0
            max_codon = ''
            for codon in codons:
                prob = self.codon_to_prob.get(codon, 0)
                if prob > max_prob:
                    max_prob = prob
                    max_codon = codon
            aa_to_max_codon[aa] = max_codon
        return aa_to_max_codon
    
    def convert(self, s):
        rna_sequence = ''.join(self.amino_acid_to_max_codon.get(aa, 'NNN') for aa in s)
        return rna_sequence

if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))


CUAAUU


### Idea of solution

Mapping Function: The get_max_codon_dict method maps each amino acid to the codon with the highest probability using the CODON_PROB dictionary.</br>

Conversion: The convert method translates each amino acid in the protein sequence to its corresponding most probable RNA codon and concatenates them into the final RNA sequence.<br/>

### Discussion

print(protein_to_rna.convert("LTPIQNRA"))<br/>
Expected result: CUAAUU

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [57]:
import random

def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Returns a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    rnd = random.uniform(0, 1)
    
    cumulative_prob = 0.0
    for event, prob in dist.items():
        cumulative_prob += prob
        if rnd < cumulative_prob:
            return event

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))


T, C, T, T, G, G, T, T, C, T, T, C, C, C, T, T, T, C, T, T, C, C, A, T, T, A, T, G, T


### Idea of solution

The random_event function selects a random event from a given probability distribution by generating a random number between 0 and 1. It then determines which event the random number falls into based on cumulative probabilities. By summing the probabilities and comparing the random number with these cumulative values, the function identifies and returns the event corresponding to the interval in which the random number falls.


### Discussion

Given the distribution {'A': 0.10, 'C': 0.35, 'G': 0.15, 'T': 0.40}, the cumulative probabilities are:

A: [0, 0.10)
C: [0.10, 0.45)
G: [0.45, 0.60)
T: [0.60, 1.00)
The function will use these intervals to determine which event to return based on the generated random number.

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [116]:
import random

CODON_PROB = {'UUU': 0.8846153846153846, 'UUC': 0.11538461538461539, 'UUA': 0.15966386554621848, 'UUG': 0.01680672268907563, 'CUU': 0.3697478991596639, 'CUC': 0.01680672268907563, 'CUA': 0.3865546218487395, 'CUG': 0.05042016806722689, 'AUU': 0.6835443037974683, 'AUC': 0.0379746835443038, 'AUA': 0.27848101265822783, 'AUG': 1.0, 'GUU': 0.47058823529411764, 'GUC': 0.025210084033613446, 'GUA': 0.453781512605042, 'GUG': 0.05042016806722689}

AA_TO_RNA = {'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UAG'], 'UGA': ['40285)'], '63237)': ['('], 'UGG': ['32109)'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Returns a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    rnd = random.uniform(0, 1)
    
    cumulative_prob = 0.0
    for event, prob in dist.items():
        cumulative_prob += prob
        if rnd < cumulative_prob:
            return event

class ProteinToRandomRNA(object):
    def __init__(self):
        self.codon_probs = CODON_PROB
        self.aa_to_rna = AA_TO_RNA

    def __get_random_codon_from_distribution(self, aa):
        candidate_codons = self.aa_to_rna[aa]
        candidate_probs = {codon: self.codon_probs.get(codon, 1 / len(candidate_codons)) for codon in candidate_codons}
        total_prob = sum(candidate_probs.values())
        candidate_probs = {codon: prob / total_prob for codon, prob in candidate_probs.items()}
        return random_event(candidate_probs)

    def convert(self, s):
        random_rna_sequence = [self.__get_random_codon_from_distribution(aa) for aa in s]
        return "".join(random_rna_sequence)

if __name__ == '__main__':
    protein_to_random_rna = ProteinToRandomRNA()
    print(protein_to_random_rna.convert("LTPIQNRA"))


UUAACUCCGAUUCAAAAUAGAGCG


### Idea of solution

To address the problem of converting a protein sequence to a random RNA sequence based on codon adaptation probabilities, we need to ensure we correctly handle the amino acid to codon mapping and the associated probabilities. We'll start by defining the codon probabilities and the amino acid to RNA mappings. Next, we'll create a function to randomly select an event based on these probabilities. The `ProteinToRandomRNA` class will include methods to initialize with these mappings, normalize the probabilities, and handle missing codons. The `convert` method will iterate through the protein sequence and convert each amino acid to a corresponding codon, forming the RNA sequence. By testing with a sample protein sequence, we can verify the correct implementation of this process. This approach ensures the selection of codons is based on their actual probabilities and handles cases where specific codons might be missing from the input data.

### Discussion

fill in 

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [77]:
def sliding_window(s, k):
    """
    This function returns a generator that iterates over all
    starting positions of a k-window in the sequence.
    For each starting position, the generator yields the nucleotide frequencies
    in the window as a dictionary.
    """
    # Initialize the window frequencies
    freq = {'A': 0, 'C': 0, 'G': 0, 'T': 0}
    
    # Fill frequencies for the initial window
    for i in range(k):
        if i < len(s):
            freq[s[i]] += 1
    
    # Yield frequencies for the initial window
    yield freq.copy()
    
    # Slide the window
    for i in range(k, len(s)):
        freq[s[i - k]] -= 1
        freq[s[i]] += 1
        yield freq.copy()

if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)


{'A': 0, 'C': 3, 'G': 0, 'T': 1}
{'A': 0, 'C': 3, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}
{'A': 0, 'C': 2, 'G': 0, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}


### Idea of solution

The sliding_window generator iterates over a DNA sequence with a fixed-size window, calculating and updating base frequencies efficiently. It starts by initializing frequency counts for the first window. As the window slides one position at a time, it updates the frequencies by decrementing the count of the nucleotide that leaves the window and incrementing the count of the nucleotide that enters. This approach ensures that each update operation is performed in constant time, yielding nucleotide frequency dictionaries for each window position.

### Discussion

For the provided example sequence "TCCCGACGGCCTTGCC" and a window size of 4, the output will be dictionaries of nucleotide frequencies for each window position as it slides across the sequence. Each dictionary represents the counts of 'A', 'C', 'G', and 'T' in that window.

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [78]:
def context_list(s, k):
    context_dict = {}
    
    # Iterate over the sequence to extract k-mers and their following characters
    for i in range(len(s) - k):
        kmer = s[i:i + k]
        following_char = s[i + k]
        
        if kmer not in context_dict:
            context_dict[kmer] = ""
        
        context_dict[kmer] += following_char

    return context_dict

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)


{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution

The function context_list iterates through the DNA sequence and extracts all k-mers of length k. For each k-mer, it tracks the characters that immediately follow this k-mer. The result is a dictionary where each k-mer is a key, and the associated value is a string of all characters that appear directly after this k-mer in the sequence.

### Discussion

Expected reusult:<br/>
{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [79]:
from collections import defaultdict

def context_probabilities(s, k):
    if k == 0:
        freq = defaultdict(int)
        for char in s:
            freq[char] += 1
        total = sum(freq.values())
        probabilities = {char: count / total for char, count in freq.items()}
        return probabilities
    
    context_dict = context_list(s, k)
    
    probabilities = {}
    for context, following_chars in context_dict.items():
        freq = defaultdict(int)
        for char in following_chars:
            freq[char] += 1
        total = len(following_chars)
        probabilities[context] = {char: count / total for char, count in freq.items()}
    
    return probabilities

if __name__ == '__main__':
    s = "ATGATATCATCGACGATGTAG"
    print("k=0 probabilities:")
    print(context_probabilities(s, 0))
    print("\nk=2 probabilities:")
    print(context_probabilities(s, 2))


k=0 probabilities:
{'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}

k=2 probabilities:
{'AT': {'G': 0.4, 'A': 0.2, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5}, 'GA': {'T': 0.6666666666666666, 'C': 0.3333333333333333}, 'TA': {'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}


### Idea of solution

To calculate context probabilities for a given DNA sequence, the context_probabilities function first computes the frequency of each character following each k-mer using the context_list function. For k=0, it counts the frequency of each character directly in the sequence and converts these counts into probabilities. For k>0, it counts how often each character follows every possible k-mer and then converts these counts into probabilities by dividing the count of each character by the total count of characters following that k-mer.

### Discussion

k=0 probabilities:
{'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}

k=2 probabilities:
{'AT': {'G': 0.4, 'A': 0.2, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5}, 'GA': {'T': 0.6666666666666666, 'C': 0.3333333333333333}, 'TA': {'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [80]:
import random

class MarkovChain:
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth

    def generate(self, n, seed=None):
        if seed is not None:
            random.seed(seed)

        sequence = ''.join(random.choices(list(self.zeroth.keys()), weights=self.zeroth.values(), k=self.k))
        
        while len(sequence) < n:
            context = sequence[-self.k:]
            if context in self.kth:
                next_char_probs = self.kth[context]
            else:
                next_char_probs = self.zeroth
            next_char = random_event(next_char_probs)
            sequence += next_char
        
        return sequence

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10    
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

GGTAGTATCG


### Idea of solution

The MarkovChain class generates random DNA sequences using a $k$-th order Markov chain model. It starts by initializing the sequence with a random selection based on zero-order probabilities. For each subsequent character, it looks at the last k characters (the context) and uses the $k$-th order probability distribution to determine the next character. If the context is not found, it falls back on the zero-order model. This process continues until the sequence reaches the desired length. The class thus produces sequences that reflect the statistical dependencies encoded in the provided $k$-th order context and zero-order models.

### Discussion

Expected output: GGTAGTATCG

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [88]:
from collections import defaultdict
from itertools import product
import random

def context_pseudo_probabilities(s, k):
    pseudo_count = 1
    kmer_counts = defaultdict(lambda: defaultdict(lambda: pseudo_count))
    kmer_total = defaultdict(lambda: pseudo_count * (4 ** len(s)))
    
    for i in range(len(s) - k):
        kmer = s[i:i+k]
        next_char = s[i+k]
        kmer_counts[kmer][next_char] += 1
        kmer_total[kmer] += 1

    kth_probabilities = {}
    for kmer, counts in kmer_counts.items():
        total = kmer_total[kmer]
        kth_probabilities[kmer] = {char: count / total for char, count in counts.items()}

    zeroth_counts = defaultdict(lambda: pseudo_count)
    for char in s:
        zeroth_counts[char] += 1
    total_chars = len(s) + len(zeroth_counts) * pseudo_count
    zeroth_probabilities = {char: count / total_chars for char, count in zeroth_counts.items()}

    return kth_probabilities, zeroth_probabilities

def random_event(dist):
    events, probs = zip(*dist.items())
    return random.choices(events, probs)[0]

class MarkovChain:
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth

    def generate(self, n, seed=None):
        if seed is not None:
            random.seed(seed)
        
        sequence = ''.join(random.choices(list(self.zeroth.keys()), weights=self.zeroth.values(), k=self.k))
        
        while len(sequence) < n:
            context = sequence[-self.k:]
            if context in self.kth:
                next_char_probs = self.kth[context]
            else:
                next_char_probs = self.zeroth
            next_char = random_event(next_char_probs)
            sequence += next_char
        
        return sequence

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth, zeroth = context_pseudo_probabilities(s, k)
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    print("\n", MarkovChain(zeroth, kth, k).generate(20))


zeroth: {'A': 0.32, 'T': 0.28, 'G': 0.24, 'C': 0.16}
AT: {'G': 6.821210263289207e-13, 'A': 4.5474735088594713e-13, 'C': 6.821210263289207e-13}
TG: {'A': 4.547473508862573e-13, 'T': 4.547473508862573e-13}
GA: {'T': 6.821210263292309e-13, 'C': 4.5474735088615393e-13}
TA: {'T': 4.547473508862573e-13, 'G': 4.547473508862573e-13}
TC: {'A': 4.547473508862573e-13, 'G': 4.547473508862573e-13}
CA: {'T': 4.547473508863607e-13}
CG: {'A': 6.82121026329386e-13}
AC: {'G': 4.547473508863607e-13}
GT: {'A': 4.547473508863607e-13}

 ATCATCATGTATGATCATCG


### Idea of solution

To generate a DNA sequence using a higher-order Markov chain with pseudo counts, first compute k-mer probabilities with pseudo counts to handle unseen k-mers. Initialize the sequence using zero-order probabilities and then iteratively extend it by sampling characters based on the current k-mer context. Use the random_event function to select characters according to their probabilities. If a k-mer is not present in the training data, fall back to zero-order probabilities to ensure all possible sequences can be generated. This approach ensures a more robust and flexible sequence generation even with incomplete training data.

### Discussion

fill in 

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [91]:
import numpy as np
from collections import defaultdict

def context_pseudo_probabilities(s, k):
    pseudo_count = 1
    kmer_counts = defaultdict(lambda: defaultdict(lambda: pseudo_count))
    kmer_total = defaultdict(lambda: pseudo_count * (4 ** len("ACGT")))
    
    for i in range(len(s) - k):
        kmer = s[i:i+k]
        next_char = s[i+k]
        kmer_counts[kmer][next_char] += 1
        kmer_total[kmer] += 1

    kth_probabilities = {}
    for kmer, counts in kmer_counts.items():
        total = kmer_total[kmer]
        kth_probabilities[kmer] = {char: count / total for char, count in counts.items()}

    zeroth_counts = defaultdict(lambda: pseudo_count)
    for char in s:
        zeroth_counts[char] += 1
    total_chars = len(s) + len(zeroth_counts) * pseudo_count
    zeroth_probabilities = {char: count / total_chars for char, count in zeroth_counts.items()}

    return zeroth_probabilities, kth_probabilities

class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
    
    def probability(self, s):
        prob = 1.0
        
        for char in s[:self.k]:
            if char in self.zeroth:
                prob *= self.zeroth[char]
            else:
                return 0.0 
        
        for i in range(len(s) - self.k):
            kmer = s[i:i+self.k]
            next_char = s[i+self.k]
            
            if kmer in self.kth:
                kmer_probs = self.kth[kmer]
                if next_char in kmer_probs:
                    prob *= kmer_probs[next_char]
                else:
                    return 0.0
            else:
                return 0.0
        
        return prob

if __name__ == '__main__':
    k = 2
    zeroth, kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    mc = MarkovProb(k, zeroth, kth)
    s = "ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence '{s}' is {mc.probability(s)}")


Probability of sequence 'ATGATATCATCGACGATGTAG' is 1.7169164160343665e-40


### Idea of solution

To calculate the probability of a DNA sequence using a $k$-th order Markov chain, the MarkovProb class first initializes the probability of the sequence to 1. It computes the probability for the initial $k$ characters using zero-order probabilities and then iteratively calculates the probability for subsequent characters based on the $k$-th order model. If any k-mer or character isn't present in the training data, the method returns 0, ensuring that the model can handle unseen sequences. This approach uses pseudo counts to smooth the data and handle cases where some k-mers or characters are not observed during training, thus providing a valid probability estimate for any sequence.

### Discussion

Probability of sequence 'ATGATATCATCGACGATGTAG' is 1.7169164160343665e-40


With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [93]:
from collections import defaultdict
from itertools import product
import numpy as np

def context_pseudo_probabilities(s, k):
    pseudo_count = 1
    kmer_counts = defaultdict(lambda: defaultdict(lambda: pseudo_count))
    kmer_total = defaultdict(lambda: pseudo_count * (4 ** k))
    
    for i in range(len(s) - k):
        kmer = s[i:i+k]
        next_char = s[i+k]
        kmer_counts[kmer][next_char] += 1
        kmer_total[kmer] += 1

    kth_probabilities = {}
    for kmer, counts in kmer_counts.items():
        total = kmer_total[kmer]
        kth_probabilities[kmer] = {char: count / total for char, count in counts.items()}

    zeroth_counts = defaultdict(lambda: pseudo_count)
    for char in s:
        zeroth_counts[char] += 1
    total_chars = len(s) + len(zeroth_counts) * pseudo_count
    zeroth_probabilities = {char: count / total_chars for char, count in zeroth_counts.items()}

    return zeroth_probabilities, kth_probabilities

class MarkovLog:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
    
    def log_probability(self, s):
        log_prob = 0.0
        for char in s[:self.k]:
            if char in self.zeroth:
                log_prob += np.log2(self.zeroth[char])
            else:
                return -np.inf 
        
        for i in range(len(s) - self.k):
            kmer = s[i:i+self.k]
            next_char = s[i+self.k]
            
            if kmer in self.kth:
                kmer_probs = self.kth[kmer]
                if next_char in kmer_probs:
                    log_prob += np.log2(kmer_probs[next_char])
                else:
                    return -np.inf 
            else:
                return -np.inf
        
        return log_prob

if __name__ == '__main__':
    k = 2
    zeroth, kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    mc = MarkovLog(k, zeroth, kth)
    s = "ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence '{s}' is {mc.log_probability(s)}")


Log probability of sequence 'ATGATATCATCGACGATGTAG' is -60.12781564123669


### Idea of solution

To compute the log-probability of a DNA sequence using a $k$-th order Markov chain, the solution involves summing the logarithms of individual probabilities for each character in the sequence. Initialize the log-probability to 0. For the first $k$ characters, use zero-order probabilities and add their logarithms to the cumulative log-probability. For subsequent characters, use the $k$-th order probabilities based on the preceding $k$-mer context. If any character or context is missing, return negative infinity to signify zero probability. This log-transform approach effectively manages small probabilities and numerical precision issues. For example, for the sequence 'ATGATATCATCGACGATGTAG' with $k=2$, the computed log-probability is approximately -60.13.

### Discussion

Log probability of sequence 'ATGATATCATCGACGATGTAG' is -60.12781564123669


Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [95]:
from collections import defaultdict
from itertools import product

def better_context_probabilities(s, k):
    pseudo_count = 1
    kmer_counts = defaultdict(lambda: defaultdict(lambda: pseudo_count))
    kmer_total = defaultdict(lambda: pseudo_count * 4)  # 4 possible bases: A, C, G, T
    
    # Count frequencies in the sequence
    for i in range(len(s) - k):
        kmer = s[i:i+k]
        next_char = s[i+k]
        kmer_counts[kmer][next_char] += 1
        kmer_total[kmer] += 1
    
    # Compute k-th order probabilities
    kth_probabilities = {}
    for kmer, counts in kmer_counts.items():
        total = kmer_total[kmer]
        kth_probabilities[kmer] = {char: count / total for char, count in counts.items()}
    
    # Zero-order probabilities
    zeroth_counts = defaultdict(lambda: pseudo_count)
    for char in s:
        zeroth_counts[char] += 1
    total_chars = len(s) + len(zeroth_counts) * pseudo_count
    zeroth_probabilities = {char: count / total_chars for char, count in zeroth_counts.items()}
    
    return zeroth_probabilities, kth_probabilities

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    zeroth, kth = better_context_probabilities(s, k)
    print(f"Zero-order probabilities: {zeroth}")
    print(f"K-th order probabilities:")
    for kmer, probs in kth.items():
        print(f"{kmer}: {probs}")


Zero-order probabilities: {'A': 0.32, 'T': 0.28, 'G': 0.24, 'C': 0.16}
K-th order probabilities:
AT: {'G': 0.3333333333333333, 'A': 0.2222222222222222, 'C': 0.3333333333333333}
TG: {'A': 0.3333333333333333, 'T': 0.3333333333333333}
GA: {'T': 0.42857142857142855, 'C': 0.2857142857142857}
TA: {'T': 0.3333333333333333, 'G': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'G': 0.3333333333333333}
CA: {'T': 0.4}
CG: {'A': 0.5}
AC: {'G': 0.4}
GT: {'A': 0.4}


### Idea of solution

To optimize the space required for computing $k$-th order Markov chain probabilities, the better_context_probabilities function improves efficiency by focusing on counting occurrences directly rather than storing large concatenated sequences. It initializes frequency counters for $k$-mers and their subsequent characters using defaultdict with pseudo counts to handle unseen contexts. By iterating through the sequence and updating these counts, it then computes probabilities directly from the frequencies. This approach avoids the storage overhead of large concatenated sequences and efficiently calculates both zero-order and $k$-th order probabilities.

### Discussion

fill in

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [97]:
import numpy as np
from collections import defaultdict

class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.k = k
        self.zeroth, self.kth = self.compute_probabilities(s, k)
    
    def compute_probabilities(self, s, k):
        pseudo_count = 1
        
        # Zero-order probabilities
        zeroth_counts = defaultdict(lambda: pseudo_count)
        for char in s:
            zeroth_counts[char] += 1
        total_chars = len(s) + len(zeroth_counts) * pseudo_count
        zeroth_probabilities = {char: count / total_chars for char, count in zeroth_counts.items()}
        
        # k-th order probabilities
        kmer_counts = defaultdict(lambda: defaultdict(lambda: pseudo_count))
        kmer_total = defaultdict(lambda: pseudo_count * 4)
        
        for i in range(len(s) - k):
            kmer = s[i:i+k]
            next_char = s[i+k]
            kmer_counts[kmer][next_char] += 1
            kmer_total[kmer] += 1
        
        kth_probabilities = {}
        for kmer, counts in kmer_counts.items():
            total = kmer_total[kmer]
            kth_probabilities[kmer] = {char: count / total for char, count in counts.items()}
        
        return zeroth_probabilities, kth_probabilities

    def generate(self, n, seed=None):
        if seed is not None:
            np.random.seed(seed)
        
        # Start with a zero-order model to initialize the sequence
        initial_chars = list(self.zeroth.keys())
        initial_probs = list(self.zeroth.values())
        
        # Ensure probabilities sum to 1
        initial_probs = np.array(initial_probs)
        initial_probs /= initial_probs.sum()
        
        sequence = list(np.random.choice(initial_chars, p=initial_probs, size=self.k))
        
        while len(sequence) < n:
            context = ''.join(sequence[-self.k:])
            if context in self.kth:
                next_chars = list(self.kth[context].keys())
                next_probs = list(self.kth[context].values())
            else:
                next_chars = list(self.zeroth.keys())
                next_probs = list(self.zeroth.values())
            
            # Ensure probabilities sum to 1
            next_probs = np.array(next_probs)
            next_probs /= next_probs.sum()
            
            next_char = np.random.choice(next_chars, p=next_probs)
            sequence.append(next_char)
        
        return ''.join(sequence)

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))


AGTAGTATGA


### Idea of solution

In the SimpleMarkovChain class, the idea is to generate DNA sequences based on a k-th order Markov chain model with optimized memory usage. The class initializes by computing the zero-order and k-th order probabilities from the input sequence s. During sequence generation, it uses np.random.choice to select the next character based on the calculated probabilities. To ensure proper functionality, it normalizes the probabilities so they sum to 1 before sampling. This approach allows efficient sequence generation by leveraging the Markov chain's context-dependent probabilities while avoiding excessive memory usage.

### Discussion

fill in

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [98]:
def kmer_index(s, k):
    index = {}
    
    for i in range(len(s) - k + 1):
        kmer = s[i:i + k]
        if kmer not in index:
            index[kmer] = []
        index[kmer].append(i)
    
    return index

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    
    print("Using string:")
    print(s)
    print("".join([str(i % 10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d = kmer_index(s, k)
    print(dict(d))


Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}


### Idea of solution

To build a $k$-mer index, I plan to use a dictionary where each key will be a $k$-mer and the corresponding value will be a list of positions where that $k$-mer appears in the sequence. I'll slide a window of size $k$ across the sequence, and for each position, I'll extract the $k$-mer and update the dictionary. If the $k$-mer is already a key in the dictionary, I'll append the current position to its list. If it’s not in the dictionary yet, I'll create a new entry with the current position. This method ensures that I efficiently keep track of all occurrences of each $k$-mer and allows quick lookups for any given $k$-mer.

### Discussion

fill in

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [103]:
from collections import defaultdict
from itertools import product
import numpy as np
from numpy.random import choice

def codon_probabilities(rna):
    """
    Given an RNA sequence, calculates the probability of all 3-mers (codons)
    empirically based on the sequence.
    """
    codon_counts = defaultdict(int)
    total_codons = 0
    
    for i in range(len(rna) - 2):
        codon = rna[i:i+3]
        if len(codon) == 3:  # Ensure we're counting valid codons
            codon_counts[codon] += 1
            total_codons += 1
    
    return {codon: count / total_codons for codon, count in codon_counts.items()}

def kullback_leibler(p, q):
    """
    Computes the Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    """
    divergence = 0.0
    
    for event in p:
        if p[event] > 0:
            q_prob = q.get(event, 0)
            if q_prob > 0:
                divergence += p[event] * np.log2(p[event] / q_prob)
            else:
                # If q[event] == 0 but p[event] > 0, this term is infinite.
                return np.inf
    
    return divergence

def generate_rna_from_protein(protein, codon_usage):
    """
    Generate an RNA sequence from a protein sequence using given codon usage probabilities.
    """
    # List of all possible codons
    codons = [''.join(codon) for codon in product("ACGU", repeat=3)]
    
    rna = []
    
    for aa in protein:
        # Example codon usage table: use the given codon_usage for this example
        codon_probs = np.array([codon_usage.get(codon, 1 / len(codons)) for codon in codons])
        codon_probs /= codon_probs.sum()  # Normalize probabilities to sum to 1
        chosen_codon = np.random.choice(codons, p=codon_probs)
        rna.append(chosen_codon)
    
    return ''.join(rna)

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY")  # List of amino acids
    n = 10000
    
    # Generate a random protein sequence
    protein = "".join(np.random.choice(aas, n))
    
    codon_usage = {''.join(codon): 1 / 64 for codon in product("ACGU", repeat=3)}  # Example uniform distribution
    rna_sequence = generate_rna_from_protein(protein, codon_usage)
    
    # Calculate codon probabilities of the RNA sequence
    cp_predicted = codon_probabilities(rna_sequence)
    
    # Assume uniform codon probabilities for comparison
    cp_uniform = {''.join(codon): 1 / 64 for codon in product("ACGU", repeat=3)}
    
    # Original codon usage probabilities (example; you would replace this with real data)
    cp_orig = {''.join(codon): 1 / 64 for codon in product("ACGU", repeat=3)}
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))


d(original || predicted) = 0.0015531910156669966
d(predicted || original) = 0.0015629421153111273

d(original || uniform) = 0.0
d(uniform || original) = 0.0

d(predicted || uniform) = 0.0015629421153111273
d(uniform || predicted) = 0.0015531910156669966


### Idea of solution

To tackle the comparison of probability distributions, we need a structured approach starting with generating an RNA sequence from a protein sequence based on codon usage probabilities. The key steps involve creating a mapping of codons to their probabilities, generating the RNA sequence by sampling codons according to these probabilities, and then computing the codon frequencies in the generated RNA. This approach ensures that we account for the distribution of codons effectively, providing a realistic model of RNA sequence synthesis based on a given protein sequence. To compare different distributions, we use the Kullback-Leibler divergence, which quantifies how one probability distribution diverges from a second, reference distribution. This measure helps in assessing the accuracy of the codon usage model by comparing it against a uniform distribution and an original distribution.

The implementation includes generating a random protein sequence, creating an RNA sequence with specified codon probabilities, and then calculating codon probabilities for both the generated RNA and a uniformly random RNA sequence. The `kullback_leibler` function is used to compute the divergence between these distributions, providing insights into how well the generated RNA sequence models the expected codon usage. By examining these divergences, we can evaluate the performance of our codon usage model and the impact of uniform randomness on the resulting RNA distributions. This approach ensures that we can critically analyze and compare different probabilistic models of RNA sequence generation.

### Discussion

fill in

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [21]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    return np.random.rand(2, 4) - 0.5
    
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p) 
        for p in get_stationary_distributions(transition)))

+0.162, -0.227, +0.388, +0.151
-0.370, +0.057, +0.198, +0.037


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [22]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    return zip([1, 10, 100, 1000, 10000], np.random.rand(5))

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[ 0.3   0.    0.7   0.  ]
 [ 0.    0.4   0.    0.6 ]
 [ 0.35  0.    0.65  0.  ]
 [ 0.    0.2   0.    0.8 ]]
Stationary distributions:
[[ 0.02421173 -0.29026212  0.1389851   0.44623574]
 [ 0.24261234  0.09071543  0.35909807 -0.47662849]]
Using [0.24, 0.09, 0.36, -0.48] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.60666836
KL divergence of stationary distribution prefix of length    10 is 0.71775085
KL divergence of stationary distribution prefix of length   100 is 0.38115273
KL divergence of stationary distribution prefix of length  1000 is 0.36116144
KL divergence of stationary distribution prefix of length 10000 is 0.22844669


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [23]:
def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

Transition probabilities are:
[[ 0.3   0.1   0.5   0.1 ]
 [ 0.2   0.3   0.15  0.35]
 [ 0.25  0.15  0.2   0.4 ]
 [ 0.35  0.2   0.4   0.05]]
Equilibrium distribution:
[ 0.05549613 -0.0618828  -0.3045492   0.11913485]

Using [-0.3337754   0.45068768 -0.10300145 -0.20088342] as initial distribution:
kl-divergence   empirical distribution
0.25023050057   [ 0.2243623   0.15184879 -0.36068593  0.0442362 ]
0.85018673369   [-0.27420015 -0.1344076  -0.08342149  0.10517491]
0.69016371310   [-0.11457606  0.27115112  0.16359241 -0.1280264 ]
0.14692236781   [ 0.07706555  0.23608696  0.01647088 -0.08926859]
0.98884289212   [-0.06341222  0.45084425  0.3957673  -0.40553139]

Using [-0.14182502  0.21310283  0.21065106  0.45942999] as initial distribution:
kl-divergence   empirical distribution
0.09700875402   [ 0.35101425 -0.18876535  0.25445247 -0.20776396]
0.51559517427   [ 0.31812104 -0.03010157 -0.2113726  -0.49508434]
0.92023091580   [ 0.40268876 -0.22995087 -0.42421415 -0.1464154 ]
0.32096833450  

### Idea of solution

fill in

### Discussion
fill in